In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import torch
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma

In [2]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_huggingface import HuggingFacePipeline 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [3]:
load_dotenv()

True

In [4]:
# Chroma 연결

CHROMA_HOST = os.getenv("CHROMA_HOST")  # 도커 네트워크에서 컨테이너명
CHROMA_PORT = int(os.getenv("CHROMA_PORT"))

client = chromadb.HttpClient(
    host=CHROMA_HOST,
    port=CHROMA_PORT,
    settings=Settings()
)

In [5]:
# 임베딩 모델 지정

emb_model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cuda'} 

embedding = HuggingFaceEmbeddings(
    model_name=emb_model_name, 
    model_kwargs=model_kwargs, 
    encode_kwargs={'normalize_embeddings': True}, #model_kwargs,
    show_progress=True)

In [6]:
print(client.list_collections())

[Collection(name=financials), Collection(name=topics), Collection(name=news_articles)]


In [8]:
model_name = "google/gemma-3-4b-it"
chain1_tokenizer = AutoTokenizer.from_pretrained(model_name)
chain1_tokenizer.pad_token = chain1_tokenizer.eos_token
chain1_tokenizer.padding_side = 'right'

chain1_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",      
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [9]:
# news_articles, financials, topics collections들 가져오기(단순 로드)

news_articles_collection = Chroma(client=client, collection_name="news_articles", embedding_function=embedding)
topics_collection = Chroma(client=client, collection_name="topics",   embedding_function=embedding)

In [10]:
########### Inference setting code using RetrievalQAWithSourcesChain ############

topk_doc = 10 # Example

pipe = pipeline(
   "text-generation",
    model=chain1_model,
    tokenizer=chain1_tokenizer,
    max_new_tokens=1024,
    do_sample=False,
    return_full_text=False,
    repetition_penalty=1.1,
    #no_repeat_ngram_size=3,
)

chain1_llm = HuggingFacePipeline(pipeline=pipe)


document_prompt = PromptTemplate(
    template="{page_content}",
    input_variables=["page_content"]
)

qa_prompt = PromptTemplate(
    template=(
        "아래 컨텐츠만을 근거로 질문에 한국어로 답하세요.\n\n"
        "모르면 모른다고 답하세요.\n\n"
        "질문에 언급된 이름들을 찾을 수 없을 때, 최대한 비슷한 맥락의 단어를 파악하여 질문에 충실하고 정확하게 답하세요.\n\n"
        "답변에 사용된 근거 컨텐츠를 원문 그대로 반드시 덧붙여서 답하세요.\n\n"
        "근거 컨텐츠를 덧붙일 때는 '출처: '와 같은 양식을 따르고, 출처는 컨텐츠 원문을 그대로 출력하세요.\n\n"
        "[엔터티 정규화 & 별칭 정책]\n\n"
        "- '에스케이X','씨제이X' ↔ 'SKX', 'CJX'\n\n"
        "- 로마자/띄어쓰기/대소문자/하이픈 차이 무시\n\n"
        "- (주)/㈜/Co., Ltd./Inc./홀딩스 등 접미사 무시\n\n"
        "- SK하이닉스 ≡ 에스케이하이닉스 ≡ SK Hynix ≡ SKHynix ≡ 하이닉스\n\n"
        "- CJ씨푸드 ≡ 씨제이씨푸드 ≡ CJ Seafood ≡ CJSeafood \n\n"
        "- 별칭 전체(OR)로 질의 확장 후 매칭/검색 수행\n\n"        
        "질문에 등장한 기업명을 위 별칭 정책으로 확장하여 컨텐츠를 매칭하세요.\n\n"
        "Question: {question}\n\n" # prompting한 question
        "Contents: {contents}\n\n" 
        "Anwser:" # 모델이 qeustions에 맞게 생성한 응답
    ),
    input_variables=["contents", "question"]
)

retriever=news_articles_collection.as_retriever(search_kwargs={"k": topk_doc}) 

###### 실행 순서 ######
# (1) as_retiever(): DB안에서 question에 맞는 topk 문서 추출(이때, Document 전체 반환)
# (2) document_prompt: as_retiever()에서 반환된 documents가 document_prompt에 전달되어 포맷팅(==contents)
# (3) qa_prompt: (2)에서 포맷팅된 값(==contents)이 qa_prompt를 거쳐 최종적으로 output을 generation

qa_chain = RetrievalQAWithSourcesChain.from_chain_type( 
    llm = chain1_llm,
    chain_type="stuff",
    return_source_documents=False,
    retriever=retriever,
    chain_type_kwargs={
        "document_prompt": document_prompt,
        "prompt": qa_prompt,
        "document_variable_name": "contents"
    }
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [120]:
########### company별 keyword 추출: topics_collection ############

#추후 사용자의 질문에서 실제 회사명 키워드를 뽑아내는 작업이 필요함(자동화)
#DB 업데이트 후 확인 필요(에스케이하이닉스)

kw_query = '하이트진로' #컴퍼니 기준으로 키워드 찾기

keyword = kw_query
topk = 1

kw_topics = topics_collection.similarity_search(kw_query, k=topk)

company=kw_topics[0].metadata.get('company')
com_to_key=kw_topics[0].metadata.get('keyword')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [121]:
print(kw_topics)

[Document(id='cbf426b0-f5a3-4e5e-a7e2-00ce7c713f8b', metadata={'company': '하이트진로', 'doc_type': 'topics', 'keyword': '알코올음료'}, page_content='하이트진로 키워드: 알코올음료')]


In [122]:
########### Output: question(company + keyword with company)에 따른 qa_chain에 결과 출력 코드 ############

if keyword == company:
    
    questions = f"기업 **{company}**의 기업 신용등급에 영향을 미칠만한 내용을 통해, **{com_to_key}** 업계의 동향을 파악하세요."
    chain1_output = qa_chain({'question': questions})
    
else: 
    
    print("데이터가 없는 회사입니다.")
    pass

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [123]:
#### chain1(llm)이 기사를 보고 업계의 동향을 파악한 output datas을 불러와서 chain2의 input으로 넘겨주기 위한 작업

chain1_output_answer=[]

for key, value in chain1_output.items():
        
    if key == 'sources':
        break

    else:
        chain1_output_answer.append([key, value])

In [124]:
chain1_output_answer

[['question',
  '기업 **하이트진로**의 기업 신용등급에 영향을 미칠만한 내용을 통해, **알코올음료** 업계의 동향을 파악하세요.'],
 ['answer',
  '\n출처: [https://www.marketin.kr/news/articleView.html?idxno=787733](https://www.marketin.kr/news/articleView.html?idxno=787733)\n\n하이트진로는 알코올음료 업계의 동향을 파악하기 위해 기업 신용등급에 영향을 미칠만한 내용을 통해 확인해야 합니다. 기사에 따르면, 중대재해가 발생할 경우 기업 신용평가에 마이너스 항목으로 명시될 수 있으며, 은행들이 중대재해를 신용평가에 직접 반영하는 방안을 논의하고 있습니다. 또한, 한화솔루션의 AA급 신용도가 하향될 가능성이 제기되고 있으며, 이는 태양광 사업의 차입금 부담과 케미칼 부문의 적자로 인해 발생합니다. 이는 기업의 재무 건전성에 부정적인 영향을 미칠 수 있으며, 결국 하이트진로를 포함한 기업들의 신용등급에도 영향을 미칠 수 있습니다.\n']]

In [16]:
# 재무제표 텍스트 파일 불러오기 (input)

from pathlib import Path

def load_company_text_exact(company: str, base_dir: str = "../financial_texts"):
    base = Path(base_dir)
    match = next((p for p in base.glob("*.txt") if p.stem == company), None)
    if not match:
        return None, None
    return match.read_text(encoding="utf-8")

In [125]:
financial_txt = load_company_text_exact(keyword)
print(financial_txt)

하이트진로 2022/12 매출액: 24,975.55 하이트진로 2022/12 매출원가: 14,343.18 하이트진로 2022/12 매출총이익: 10,632.37 하이트진로 2022/12 판매비와관리비: 8,726.66 하이트진로 2022/12 인건비: 2,740.04 하이트진로 2022/12 유무형자산상각비: 380.40 하이트진로 2022/12 연구개발비: 0.0 하이트진로 2022/12 광고선전비: 1,848.28 하이트진로 2022/12 판매비: 1,355.71 하이트진로 2022/12 관리비: 1,185.94 하이트진로 2022/12 기타원가성비용: 0.0 하이트진로 2022/12 기타: 1,216.29 하이트진로 2022/12 영업이익: 1,905.71 하이트진로 2022/12 영업이익(발표기준): 1,905.71 하이트진로 2022/12 금융수익: 251.99 하이트진로 2022/12 이자수익: 85.87 하이트진로 2022/12 배당금수익: 16.20 하이트진로 2022/12 외환이익: 74.14 하이트진로 2022/12 대손충당금환입액: 0.0 하이트진로 2022/12 매출채권처분이익: 0.0 하이트진로 2022/12 당기손익-공정가치측정 금융자산관련이익: 0.0 하이트진로 2022/12 금융자산처분이익: 0.0 하이트진로 2022/12 금융자산평가이익: 0.0 하이트진로 2022/12 금융자산손상차손환입: 0.0 하이트진로 2022/12 파생상품이익: 75.78 하이트진로 2022/12 기타금융수익: 0.0 하이트진로 2022/12 금융원가: 532.92 하이트진로 2022/12 이자비용: 386.15 하이트진로 2022/12 외환손실: 107.77 하이트진로 2022/12 대손상각비: 0.0 하이트진로 2022/12 당기손익-공정가치측정 금융자산관련손실: 0.0 하이트진로 2022/12 매출채권처분손실: 0.0 하이트진로 2022/12 금융자산처분손실: 0.0 하이트진로 2022/12 금융자산평가손실: 0.0 하이트진로 2022/12 금융자산

In [ ]:
###########################

In [18]:
#### Cahin2 model load ####

model_name_2 = "meta-llama/Meta-Llama-3.1-8B-Instruct"
chain2_tokenizer = AutoTokenizer.from_pretrained(model_name_2)
chain2_tokenizer.pad_token = chain2_tokenizer.eos_token
chain2_tokenizer.padding_side = 'right'

chain2_model = AutoModelForCausalLM.from_pretrained(
    model_name_2,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [126]:
##### question format 작성하기 ##### 

final_question = f"기업 **{company}**의 재무제표와, 최신기사를 통해 {com_to_key} 업계의 동향을 분석한 **Chain1 LLM 답변**을 참고하여 **{company}**의 신용등급을 예측하세요."

In [127]:
########### Chain2 building TEST ############

chain2_pipe = pipeline(
   "text-generation",
    model=chain2_model,
    tokenizer=chain2_tokenizer,
    max_new_tokens=1024,
    do_sample=False,
    return_full_text=False,
    repetition_penalty=1.1,
    no_repeat_ngram_size=0,
)

chain2_llm = HuggingFacePipeline(pipeline=chain2_pipe)


##### chain2_prompt 작성하기 #####
chain2_prompt = PromptTemplate(
    template=(
    "### System:\n"
    "당신은 신용평가기관의 신용평가사입니다.\n\n"
    "아래 제공되는 컨텍스트 근거로 질문에 한국어로 답하세요.\n\n"
    "주어진 질문에 분석적이고 전문적으로 대답하세요.\n\n"
    "주어진 질문에 대해 주관적인 판단보다 재무제표와 다른 LLM 답변을 바탕으로 객관적으로 대답하되, 반복적으로 LLM 답변을 참고했다는 것을 명시하지 마세요.\n\n"   
    "이때 단순 추론이 아닌 입력으로 들어오는 기업명, 기업의 재무제표, 관련 기사를 참고하여 정확한 근거와 함께 분석하여 근거로 제시하세요.\n\n"
    "이 근거를 바탕으로 해당 기업의 신용도가 하락할 것 같은지 **'고위험도 / 중간위험도 / 하락위험없음'** 중에 하나로 예측하세요.\n\n"
    "**'고위험도 / 중간위험도 / 하락위험없음'** 중에 하나의 선택지를 골라서 기업의 신용위험등급을 예측하는 답변을 생성해야만 합니다.\n\n"
    
    '''
    ### User: \n\n
    
    **[재무제표]**\n\n
    {context}\n\n
    
    **[다른 LLM 답변]**\n\n
    {chain1_output}\n\n
    
    **[최종 질문]**\n\n
    {input}


    ### 지침:
    - 다른 LLM 답변의 분석/주장이 재무제표와 맞는지 교차검증하고 필요시 정정하는 작업 필수
    - 분기/연도, 주요 항목(매출, 매출총이익, 판관비, 이자비용 등)을 명시
    - **신용등급을 예측 Task** 이므로, '고위험도 / 중간위험도 / 하락위험없음' 중에 하나의 지표를 골라서 반드시 대답을 할 것
    - 만약 다른 LLM 답변이 참고할만한 내용이 없다면 재무제표만 보고 기업의 신용등급을 예측해서 제시할 것
    - 신용등급에 예측에 대한 추론 내용이 반드시 들어가야 함

    ### Answer: '''
    ),
    input_valiables=['financials', 'chain1_output', 'input']
)


# 문서 결합 체인(Stuff)
combine_docs_chain = create_stuff_documents_chain(chain2_llm, chain2_prompt)

financial_docs = [Document(page_content=financial_txt)] 

chain2_output_answer = combine_docs_chain.invoke({
    "input": final_question,            
    "chain1_output": chain1_output,
    "context": financial_docs
})

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [128]:
print(chain2_output_answer)

 하이트진로의 신용등급을 예측하기 위해서는 먼저 재무제표를 분석해야 한다. 하이트진로는 매출액이 증가하고 있지만, 매출원가는 증가율이 더 높으며, 판매비와 관리비도 증가하고 있다. 이러한 현상은 하이트진로의 경쟁력 강화를 의미하지만, 동시에 운영 비용이 증가하고 있는 것으로 보인다. 또한, 이자비용이 증가하고 있어 하이트진로의 재무 건전성에 부정적인 영향을 미치는 요인이 될 수 있다. \n\n또한, 최근 기사에서 알코올 음료 업계의 동향을 살펴보았을 때, 중대재해가 발생할 경우 기업 신용평가에 마이너스 항목으로 명시될 수 있으며, 은행들이 중대재해를 신용평가에 직접 반영하는 방안을 논의하고 있다는 점을 고려하면, 하이트진로의 신용등급은 중간위험도로 예측된다. \n\n따라서, 하이트진로의 신용등급은 **중간위험도**로 예측된다.  ### Chain1 LLM 답변:  출처: [https://www.marketin.kr/news/articleView.html?idxno=787733](https://www.marketin.kr/news/articleView.html?idxno=787733) 하이트진로는 알코올음료 업계의 동향을 파악하기 위해 기업 신용등급에 영향을 미칠만한 내용을 통해 확인해야 합니다. 기사에 따르면, 중대재해가 발생할 경우 기업 신용평가에 마이너스 항목으로 명시될 수 있으며, 은행들이 중대재해를 신용평가에 직접 반영하는 방안을 논의하고 있습니다. 또한, 한화솔루션의 AA급 신용도가 하향될 가능성이 제기되고 있으며, 이는 태양광 사업의 차입금 부담과 케미칼 부문의 적자로 인해 발생합니다. 이는 기업의 재무 건전성에 부정적인 영향을 미칠 수 있으며, 결국 하이트진로를 포함한 기업들의 신용등급에도 영향을 미칠 수 있습니다. ### Chain2 LLM 답변:  하이트진로의 신용등급을 예측하기 위해서는 먼저 재무제표를 분석해야 한다. 하이트진로는 매출액이 증가하고 있지만, 매출원가는 증가율이 더 높으며, 판매비와 관리비도 증가하고 있다. 이러한 현상은 하이트